## Setup

In [ ]:
!pip install httpfs -q
!pip install duckdb --upgrade --pre

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.1 MB/s eta 0:00:00


In [ ]:
import duckdb

In [ ]:
con = duckdb.connect()
#cur = con.cursor()

In [ ]:
con.execute('INSTALL httpfs;')
con.execute('LOAD httpfs;')
con.execute("SET s3_region='us-west-2';")

Schemas:
https://github.com/aws-solutions-library-samples/guidance-for-digital-assets-on-aws/blob/main/analytics/schema.md

### Table token_transfers

This table captures ERC-20 token transfers.

Field | Type | Description
--- | --- | ---
date | string | Partition column (YYYY-MM-DD)
token_address | string | Unique token address
from_address | string | Sender address
to_address | string | Recipient address
value | double | Amount of token
transaction_hash | string | Transaction hash of the block
log_index | bigint | Logs index position in the transaction
block_timestamp | timestamp | The unix timestamp for when the block was collated
block_number | bigint | Block number where this transaction was in
block_hash | string | Hash of the block

### Single day of data

In [ ]:
duckdb.read_parquet('s3://aws-public-blockchain/v1.0/eth/token_transfers/date=2022-01-01/*')

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌──────────────────────┬──────────────────────┬───┬─────────────────────┬────────────┬──────────────────────┐
│    token_address     │     from_address     │ … │   block_timestamp   │    date    │    last_modified     │
│       varchar        │       varchar        │   │      timestamp      │  varchar   │      timestamp       │
├──────────────────────┼──────────────────────┼───┼─────────────────────┼────────────┼──────────────────────┤
│ 0xdbdb4d16eda451d0…  │ 0x0000000000000000…  │ … │ 2022-01-01 15:19:02 │ 2022-01-01 │ 2022-09-21 13:05:0…  │
│ 0xa0b86991c6218b36…  │ 0x0000000000000000…  │ … │ 2022-01-01 17:40:53 │ 2022-01-01 │ 2022-09-21 13:25:2…  │
│ 0x8fac8031e079f409…  │ 0x0000000000000000…  │ … │ 2022-01-01 14:48:55 │ 2022-01-01 │ 2022-09-21 13:00:4…  │
│ 0x95ad61b0a150d792…  │ 0x0000000000000000…  │ … │ 2022-01-01 17:30:25 │ 2022-01-01 │ 2022-09-21 13:23:4…  │
│ 0xb088f8926ae002b7…  │ 0x0000000000000000…  │ … │ 2022-01-01 16:42:26 │ 2022-01-01 │ 2022-09-21 13:16:5…  │
│ 0x6bba31

### *All data (don't bother)*

In [ ]:
#all_data = duckdb.read_parquet('s3://aws-public-blockchain/v1.0/eth/token_transfers/*/*')

In [ ]:
#duckdb.sql("SELECT COUNT(token_address) FROM all_data;").show() #still didn't finish after 10 mins

In [ ]:
#all_data.create('token_transfers') #will this crash? creates table
#RAM survives, but 70 GB disk space after 3 hours; not enough.

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
#con = duckdb.connect(database=':default:')

In [ ]:
#CREATE TABLE test as...

In [ ]:
# con.execute("SELECT COUNT(token_address) FROM all_data WHERE date ILIKE '%2022-01%';")

### One month of data

In [ ]:
test_data = duckdb.read_parquet('s3://aws-public-blockchain/v1.0/eth/token_transfers/date=2022-01*/*')

In [ ]:
con.execute("SELECT COUNT(token_address) FROM test_data;").fetchall()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

[(26991109,)]

### One year of data

In [ ]:
test_data = duckdb.read_parquet('s3://aws-public-blockchain/v1.0/eth/token_transfers/date=2022*/*')
con.execute("SELECT COUNT(token_address) FROM test_data;").fetchall()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

[(374610256,)]

### Under half a decade of data

In [ ]:
test_data = duckdb.read_parquet('s3://aws-public-blockchain/v1.0/eth/token_transfers/date=202*/*')
con.execute("SELECT COUNT(token_address) FROM test_data;").fetchall()
#took 9.5 mins to run
# 1,179,262,670 rows of data

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

[(1179262670,)]